In [12]:
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
from PIL import Image

ROI_WIDTH = 200
ROI_HEIGHT = 100


2.1.0


In [13]:
train_dir = 'C:/Users/sunmu/OneDrive/Desktop/auto_driving/cnnimages/train'
test_dir = 'C:/Users/sunmu/OneDrive/Desktop/auto_driving/cnnimages/test'

In [14]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
)

test_datagen = ImageDataGenerator(
    rescale=1./255,
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(ROI_WIDTH,ROI_HEIGHT),
    batch_size=4,
    class_mode='binary',
    )

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(ROI_WIDTH,ROI_HEIGHT),
    batch_size=4,
    class_mode='binary'
  )


Found 259 images belonging to 2 classes.
Found 85 images belonging to 2 classes.


In [15]:
# CNN 모델 정의
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(ROI_WIDTH,ROI_HEIGHT, 3)),
    layers.MaxPooling2D((3, 3)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((3, 3)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1,activation='sigmoid')
])



In [16]:
# 모델 컴파일
model.compile(optimizer='adam',
              loss=tf.keras.losses.binary_crossentropy,
              metrics=['accuracy'])

In [17]:
history = model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // train_generator.batch_size,
    epochs=1,
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size,
)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 64 steps, validate for 21 steps
64/64 [==============================] - 3s 41ms/step - loss: 0.4196 - accuracy: 0.8078 - val_loss: 0.0211 - val_accuracy: 0.9881


In [21]:
import numpy as np

img_path = 'C:/Users/sunmu/OneDrive/Desktop/auto_driving/cnnimages/train/red/95.jpg'
frame = cv2.imread(img_path)
gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # 흑백으로 변환
resized_frame = cv2.resize(gray_frame, (ROI_HEIGHT,ROI_WIDTH))  # 크기 조정
normalized_frame = resized_frame / 255.0  # 정규화
input_data = np.expand_dims(normalized_frame, axis= 0 )  # 배치와 채널 차원 추가

print(input_data.shape)
res = model.predict(input_data)
print(res)


(1, 200, 100, 3)
[[0.9957241]]


In [18]:
model.save("my_model.h5")

In [19]:
model_path = "my_model.h5"
model = tf.keras.models.load_model(model_path)